<a href="https://colab.research.google.com/github/Lianasy/Decision-Theory-Labs/blob/main/TPR_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Імпортуємо необхідні бібліотеки
import numpy as np
import pandas as pd

# ВИБІР ПРИКЛАДНОЇ ОБЛАСТІ ТА ЗАДАЧІ

# Вхідні дані
alternatives = ["Гори", "Море", "Місто", "Дім", "Парк"]
states = ["Сонячно", "Дощ", "Сніг", "Вітер", "Хмарно", "Туман"]
decision_matrix = np.array([
    [50, 20, 30, 40, 35, 25],  # Гори
    [70, 10, 10, 30, 40, 20],  # Море
    [40, 60, 50, 30, 45, 30],  # Місто
    [30, 40, 40, 50, 35, 40],  # Дім
    [60, 30, 20, 50, 50, 30],  # Парк
])
# Перетворення у DataFrame
decision_df = pd.DataFrame(decision_matrix, index=alternatives, columns=states)

# Виведемо матрицю рішень
decision_df


,Сонячно,Дощ,Сніг,Вітер,Хмарно,Туман
Гори,50,20,30,40,35,25
Море,70,10,10,30,40,20
Місто,40,60,50,30,45,30
Дім,30,40,40,50,35,40
Парк,60,30,20,50,50,30


In [ ]:
# №1: МЕТОД МАКСИМІНУ + ЛЕКСИМІНУ

# Функція максиміну
def maximin(matrix):
    min_values = matrix.min(axis=1)  # Знаходимо найгірші результати для кожної альтернативи
    best_choice = min_values.idxmax()  # Знаходимо найкраще серед найгірших
    return best_choice, min_values

# Функція лексиміну
def leximin(matrix):
    # Сортуємо результати для кожної альтернативи
    sorted_matrix = np.sort(matrix.values, axis=1)
    leximin_values = pd.DataFrame(sorted_matrix, index=matrix.index)

    # Знаходимо найкращу альтернативу за лексикографічним порядком
    best_choice = leximin_values.apply(tuple, axis=1).idxmax()
    return best_choice, leximin_values

# Розрахунок максиміну
maximin_choice, maximin_values = maximin(decision_df)

# Перевірка: якщо в максиміні є кілька альтернатив з однаковим результатом
if (maximin_values == maximin_values.max()).sum() > 1:
    print("\nІснує кілька альтернатив з однаковим результатом у методі максиміну. Використаємо метод лексиміну.")
    leximin_choice, leximin_values = leximin(decision_df)
    print("Сортовані результати для методу лексиміну:")
    print(leximin_values)
    print(f"Рекомендована альтернатива за методом лексиміну: {leximin_choice}")
else:
    print("\nУ методі максиміну одна найкраща альтернатива.")
    print(f"Рекомендована альтернатива за методом максиміну: {maximin_choice}")



Існує кілька альтернатив з однаковим результатом у методі максиміну. Використаємо метод лексиміну.
Сортовані результати для методу лексиміну:
        0   1   2   3   4   5
Гори   20  25  30  35  40  50
Море   10  10  20  30  40  70
Місто  30  30  40  45  50  60
Дім    30  35  40  40  40  50
Парк   20  30  30  50  50  60
Рекомендована альтернатива за методом лексиміну: Дім


In [ ]:
# №2: МЕТОД МАКСИМАКСУ
def maximax(matrix):
    max_values = matrix.max(axis=1)  # Знаходимо найкращі результати для кожної альтернативи
    max_max_value = max_values.max()  # Знаходимо максимальне серед найкращих
    best_choices = max_values[max_values == max_max_value].index.tolist()  # Усі найкращі альтернативи
    return best_choices, max_values

maximax_choices, maximax_values = maximax(decision_df)
print("\nМетод максимаксу:")
print(maximax_values)
print(f"Рекомендовані альтернативи: {', '.join(maximax_choices)}")


Метод максимаксу:
Гори     50
Море     70
Місто    60
Дім      50
Парк     60
dtype: int64
Рекомендовані альтернативи: Море


In [ ]:
# №3: МЕТОД ОПТИМІЗМУ-ПЕСИМІЗМУ
def hurwicz(matrix, alpha):
    min_values = matrix.min(axis=1)
    max_values = matrix.max(axis=1)
    hurwicz_values = alpha * max_values + (1 - alpha) * min_values
    max_hurwicz_value = hurwicz_values.max()  # Максимальне значення оцінки
    best_choices = hurwicz_values[hurwicz_values == max_hurwicz_value].index.tolist()  # Усі найкращі альтернативи
    return best_choices, hurwicz_values

alpha = 0.6
hurwicz_choices, hurwicz_values = hurwicz(decision_df, alpha)
print("\nМетод оптимізму-песимізму (alpha = 0.6):")
print(hurwicz_values)
print(f"Рекомендовані альтернативи: {', '.join(hurwicz_choices)}")


Метод оптимізму-песимізму (alpha = 0.6):
Гори     38.0
Море     46.0
Місто    48.0
Дім      42.0
Парк     44.0
dtype: float64
Рекомендовані альтернативи: Місто


In [ ]:
# №4: МЕТОД МІНІМАКС ЖАЛЮ
def minimax_regret(matrix):
    regret_matrix = matrix.max(axis=0) - matrix  # Обчислюємо матрицю жалю
    max_regret = regret_matrix.max(axis=1)  # Знаходимо максимальний жаль для кожної альтернативи
    min_max_regret_value = max_regret.min()  # Мінімальний максимальний жаль
    best_choices = max_regret[max_regret == min_max_regret_value].index.tolist()  # Усі найкращі альтернативи
    return best_choices, regret_matrix, max_regret

minimax_choices, regret_matrix, max_regret = minimax_regret(decision_df)
print("\nМетод мінімакс жалю:")
print("Матриця жалю:")
print(regret_matrix)
print("Максимальний жаль для кожної альтернативи:")
print(max_regret)
print(f"Рекомендовані альтернативи: {', '.join(minimax_choices)}")


Метод мінімакс жалю:
Матриця жалю:
       Сонячно  Дощ  Сніг  Вітер  Хмарно  Туман
Гори        20   40    20     10      15     15
Море         0   50    40     20      10     20
Місто       30    0     0     20       5     10
Дім         40   20    10      0      15      0
Парк        10   30    30      0       0     10
Максимальний жаль для кожної альтернативи:
Гори     40
Море     50
Місто    30
Дім      40
Парк     30
dtype: int64
Рекомендовані альтернативи: Місто, Парк
